In [10]:
import fasttext
from sklearn.cluster import KMeans
import pandas as pd

In [17]:
# prepare dataset BELOW IS FROM FASTTEXT
#!mkdir data
#!wget -r -nc http://mattmahoney.net/dc/enwik9.zip -P data
#!unzip data/enwik9.zip -d data
#!rm data/enwik9.zip
#!perl wikifil.pl data/enwik9 > data/fil9
#!head -c 80 data/fil9


# we try with english words
# import dataset of english words and their descriptions
!mkdir data
!wget -nc -O data/english_words.csv --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1EVFrkEuXOKarjxaqLOY2nXsrG_cZCsyE'
df = pd.read_csv('data/english_words.csv')

mkdir: cannot create directory ‘data’: File exists
will be placed in the single file you specified.

File ‘data/english_words.csv’ already there; not retrieving.


<bound method NDFrame.head of                  Word                                            Meaning  \
0      A bed of roses  a phrase used to reference a situation that is...   
1            A Priori  something that is believed to be true without ...   
2            Aardvark  a large, long-eared nocturnal animal that uses...   
3              Aback                               in amazement; shocked   
4               Abaft                           toward the stern; behind   
...               ...                                                ...   
13156          Zodiac  a divided band in sky along which the sun, the...   
13157            Zone                                an area or division   
13158         Zoology  that portion of science that deals with the an...   
13159            Zoom                               to move very quickly   
13160          Zygote  the fertilized egg that results from the union...   

                                              Examples/0 